In [1]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

In [2]:
## Importing kafka libraries
#from kafka import KafkaProducer
#from kafka import KafkaConsumer
import csv
import json
# Import necessary libraries
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local").appName("ETL").getOrCreate()
spark = SparkSession.builder.appName('LinkitTest').getOrCreate()
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")
from delta.tables import DeltaTable
from pyspark.sql.functions import from_json
from pyspark.sql.types import StringType, StructType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Kafka configuration
bootstrap_servers = ['10.21.0.208:30753']
topic = 'csv_data_topic'

# Define the folder path
delta_path = "s3a://truepoc-bkt-raw/kafka/"

# Kafka SASL configurations
kafka_security_protocol = "SASL_PLAINTEXT"
kafka_sasl_mechanism = "PLAIN"
kafka_sasl_username = "user1"  # Replace with your SASL username
kafka_sasl_password = "user1"  # Replace with your SASL password

df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", bootstrap_servers)
    .option("subscribe", topic)
    .option("kafka.security.protocol", kafka_security_protocol)
    .option("kafka.sasl.mechanism", kafka_sasl_mechanism)
    .option("kafka.sasl.jaas.config",
            f"kafka.username={kafka_sasl_username};kafka.password={kafka_sasl_password};")
    .option("startingOffsets", "earliest") # You can change the starting offset as needed
    .option("includeHeaders", "true") 
    .load()
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "headers")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key: string, value: string, headers: array<struct<key:string,value:binary>>]

In [10]:
query = df.writeStream.format("console").start()
import time
time.sleep(10) # sleep 10 seconds
query.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Convert the value column from Kafka to a string
df = df.withColumn("value", df["value"].cast(StringType()))

# Parse the JSON value if your messages are in JSON format
# Modify this part according to your message structure
parsed_df = df.selectExpr("CAST(value AS STRING)").select(from_json("value", schema).alias("data")).select("data.*")

# Display the messages from Kafka topic in the console
query = (
    parsed_df
    .writeStream
    .outputMode("append")
    .format("console")
    .start()
)

# Wait for the streaming query to finish
query.awaitTermination()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[STREAM_FAILED] Query [id = d80ba6d1-81fe-487b-a762-9147730830a8, runId = 0fe3c3aa-6095-4f07-aa8a-56db6be552c4] terminated with exception: Failed to construct kafka consumer
Traceback (most recent call last):
  File "/opt/mapr/spark/spark-3.4.0/python/lib/pyspark.zip/pyspark/sql/streaming/query.py", line 201, in awaitTermination
    return self._jsq.awaitTermination()
  File "/opt/mapr/spark/spark-3.4.0/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/opt/mapr/spark/spark-3.4.0/python/lib/pyspark.zip/pyspark/errors/exceptions/captured.py", line 175, in deco
    raise converted from None
pyspark.errors.exceptions.captured.StreamingQueryException: [STREAM_FAILED] Query [id = d80ba6d1-81fe-487b-a762-9147730830a8, runId = 0fe3c3aa-6095-4f07-aa8a-56db6be552c4] terminated with exception: Failed to construct kafka consumer



In [14]:
query.close()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'StreamingQuery' object has no attribute 'close'
Traceback (most recent call last):
AttributeError: 'StreamingQuery' object has no attribute 'close'



In [13]:
# Save the DataFrame as a Delta table in the S3 bucket ---Might takes some time---
df.write.format("delta").mode("overwrite").save(delta_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'write' can not be called on streaming Dataset/DataFrame.
Traceback (most recent call last):
  File "/opt/mapr/spark/spark-3.4.0/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 510, in write
    return DataFrameWriter(self)
  File "/opt/mapr/spark/spark-3.4.0/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 945, in __init__
    self._jwrite = df._jdf.write()
  File "/opt/mapr/spark/spark-3.4.0/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/opt/mapr/spark/spark-3.4.0/python/lib/pyspark.zip/pyspark/errors/exceptions/captured.py", line 175, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: 'write' can not be called on streaming Dataset/DataFrame.

